In [2]:
import networkx as nx
import sqlite3

In [ ]:
month = '2207%'
conn = sqlite3.connect('../data/public_transportation_bh.db')
cursor = conn.cursor()

cursor.execute("""
SELECT DISTINCT endereco || ", " || num_rua FROM public_transportation_bh;
""")

nodes_name_to_id = {}
for id, linha in enumerate(cursor.fetchall()):
    nodes_name_to_id.update({linha[0]: id})

nodes_id_to_name = {v: k for k, v in nodes_name_to_id.items()}


all_lines = []
cursor.execute("""
SELECT DISTINCT linha FROM public_transportation_bh where data LIKE ?
""", (month,))
for linha in cursor.fetchall():
    all_lines.append(linha[0])


print(f"All Lines: {len(all_lines)}")

nodes = []
edges = []
for l in all_lines:

    cursor.execute("""
    SELECT DISTINCT sublinha FROM public_transportation_bh where linha = ? and data LIKE ?;
    """, (l,month,))
    sublinha = []
    for s in cursor.fetchall():
        sublinha.append(s[0])

    for sl in sublinha:

        cursor.execute("""
        SELECT data, linha, sublinha, seq, pc, endereco || ", " || num_rua, lat, lon, weekend,"0","1","2","3","4","5","6","7","8","9","10","11","12",
        "13","14","15","16","17","18","19","20","21","22","23",total
        FROM public_transportation_bh where linha = ? and sublinha = ? and weekend IS NULL and data LIKE ? and seq IS NOT NULL and pc=1 order by seq;
        """, (l,sl,month,))

        # print(f"\n\nLinha: {l} - Sublinha: {sl}")
        node_id_past = None
        for linha in cursor.fetchall():
            node_id = nodes_name_to_id[linha[5]]

            lat = linha[6]
            lon = linha[7]
            nodes.append((node_id, str(l), lat, lon))
    # print(G.nodes[u]['x'])

            # print(f'{linha[5]},{linha[7]},{linha[6]}')

            if node_id_past is None:
                node_id_past = node_id
                continue
            
            edges.append((node_id_past, node_id, {
                'date': str(linha[0]),
                'linha': str(linha[1]),
                'sub_linha': linha[2],
                'seq' : linha[3],
                'pc' : linha[4],
                'weekend' : 0,
                '0h' : linha[9],
                '1h' : linha[10],
                '2h' : linha[11],
                '3h' : linha[12],
                '4h' : linha[13],
                '5h' : linha[14],
                '6h' : linha[15],
                '7h' : linha[16],
                '8h' : linha[17],
                '9h' : linha[18],
                '10h' : linha[19],
                '11h' : linha[20],
                '12h' : linha[21],
                '13h' : linha[22],
                '14h' : linha[23],
                '15h' : linha[24],
                '16h' : linha[25],
                '17h' : linha[26],
                '18h' : linha[27],
                '19h' : linha[28],
                '20h' : linha[29],
                '21h' : linha[30],
                '22h' : linha[31],
                '23h' : linha[32],
                'passenger_total' : linha[33],
            }))
            # print(f'past id:{node_id_past} TO -> {node_id}\n\n linha {linha[1]}-sub {linha[2]}. Seq{linha[3]}. total passengers {linha[33]}')
            node_id_past = node_id
            
conn.close()


year 22 month07
All Lines: 273


In [42]:


G = nx.MultiDiGraph()

print(f"Total Nodes; {len(nodes)}")
print(f"Total Edges; {len(edges)}")

G.add_nodes_from([x[0] for x in nodes])
G.add_edges_from(edges)

for node, linha, lat, lon in nodes:

    G.nodes[node]['all_linha'] = linha
    G.nodes[node]['x'] = lon
    G.nodes[node]['y'] = lat

for l in all_lines:
    for node in G.nodes:
        G.nodes[node][str(l)] = False

for node, linha, _, _ in nodes:
    G.nodes[node][linha] = True

# Relabel the nodes
G = nx.relabel_nodes(G, nodes_id_to_name)

# Export graph
nx.write_gexf(G, f"../graph_week_{month[0:2]}_{month[2:4]}.gexf")


Total Nodes; 17701
Total Edges; 17250


# Generate Graphs individually by month

In [50]:
year = ['21','22']
month = ['01','02','03','04','05','06','07','08','09','10','11','12']
conn = sqlite3.connect('../data/public_transportation_bh.db')
cursor = conn.cursor()

cursor.execute("""
SELECT DISTINCT endereco || ", " || num_rua FROM public_transportation_bh;
""")

nodes_name_to_id = {}
for id, linha in enumerate(cursor.fetchall()):
    nodes_name_to_id.update({linha[0]: id})

nodes_id_to_name = {v: k for k, v in nodes_name_to_id.items()}

for y in year:
    for m in month:
        all_lines = []
        cursor.execute("""
        SELECT DISTINCT linha FROM public_transportation_bh where data LIKE ?
        """, (y+m+'%',))
        for linha in cursor.fetchall():
            all_lines.append(linha[0])


        print(f"All Lines of {y+m}: {len(all_lines)}")

        nodes = []
        edges = []
        for l in all_lines:

            cursor.execute("""
            SELECT DISTINCT sublinha FROM public_transportation_bh where linha = ? and data LIKE ?;
            """, (l,y+m+'%',))
            sublinha = []
            for s in cursor.fetchall():
                sublinha.append(s[0])

            for sl in sublinha:

                cursor.execute("""
                SELECT data, linha, sublinha, seq, pc, endereco || ", " || num_rua, lat, lon, weekend,"0","1","2","3","4","5","6","7","8","9","10","11","12",
                "13","14","15","16","17","18","19","20","21","22","23",total
                FROM public_transportation_bh where linha = ? and sublinha = ? and weekend = 1 and data LIKE ? and seq IS NOT NULL and pc=1 order by seq;
                """, (l,sl,y+m+'%',))

                # print(f"\n\nLinha: {l} - Sublinha: {sl}")
                node_id_past = None
                for linha in cursor.fetchall():
                    node_id = nodes_name_to_id[linha[5]]

                    lat = linha[6]
                    lon = linha[7]
                    nodes.append((node_id, str(l), lat, lon))
            # print(G.nodes[u]['x'])

                    # print(f'{linha[5]},{linha[7]},{linha[6]}')

                    if node_id_past is None:
                        node_id_past = node_id
                        continue

                    edges.append((node_id_past, node_id, {
                        'date': str(linha[0]),
                        'linha': str(linha[1]),
                        'sub_linha': linha[2],
                        'seq' : linha[3],
                        'pc' : linha[4],
                        'weekend' : 0,
                        '0h' : linha[9],
                        '1h' : linha[10],
                        '2h' : linha[11],
                        '3h' : linha[12],
                        '4h' : linha[13],
                        '5h' : linha[14],
                        '6h' : linha[15],
                        '7h' : linha[16],
                        '8h' : linha[17],
                        '9h' : linha[18],
                        '10h' : linha[19],
                        '11h' : linha[20],
                        '12h' : linha[21],
                        '13h' : linha[22],
                        '14h' : linha[23],
                        '15h' : linha[24],
                        '16h' : linha[25],
                        '17h' : linha[26],
                        '18h' : linha[27],
                        '19h' : linha[28],
                        '20h' : linha[29],
                        '21h' : linha[30],
                        '22h' : linha[31],
                        '23h' : linha[32],
                        'passenger_total' : linha[33],
                    }))
                    # print(f'past id:{node_id_past} TO -> {node_id}\n\n linha {linha[1]}-sub {linha[2]}. Seq{linha[3]}. total passengers {linha[33]}')
                    node_id_past = node_id

        G = nx.MultiDiGraph()
        print(f"Current date: {y+m}")
        print(f"Total Nodes; {len(nodes)}")
        print(f"Total Edges; {len(edges)}")

        G.add_nodes_from([x[0] for x in nodes])
        G.add_edges_from(edges)

        for node, linha, lat, lon in nodes:

            G.nodes[node]['all_linha'] = linha
            G.nodes[node]['x'] = lon
            G.nodes[node]['y'] = lat

        for l in all_lines:
            for node in G.nodes:
                G.nodes[node][str(l)] = False

        for node, linha, _, _ in nodes:
            G.nodes[node][linha] = True

        # Relabel the nodes
        G = nx.relabel_nodes(G, nodes_id_to_name)

        # Export graph
        nx.write_gexf(G, f"../graph_weekend_{y}_{m}.gexf")
            
conn.close()


All Lines of 2101: 282
Current date: 2101
Total Nodes; 21644
Total Edges; 21213
All Lines of 2102: 280
Current date: 2102
Total Nodes; 20774
Total Edges; 20344
All Lines of 2103: 282
Current date: 2103
Total Nodes; 21258
Total Edges; 20822
All Lines of 2104: 273
Current date: 2104
Total Nodes; 19003
Total Edges; 18591
All Lines of 2105: 289
Current date: 2105
Total Nodes; 21380
Total Edges; 20954
All Lines of 2106: 290
Current date: 2106
Total Nodes; 21912
Total Edges; 21480
All Lines of 2107: 291
Current date: 2107
Total Nodes; 22382
Total Edges; 21952
All Lines of 2108: 291
Current date: 2108
Total Nodes; 22846
Total Edges; 22414
All Lines of 2109: 290
Current date: 2109
Total Nodes; 23035
Total Edges; 22604
All Lines of 2110: 282
Current date: 2110
Total Nodes; 23223
Total Edges; 22786
All Lines of 2111: 283
Current date: 2111
Total Nodes; 23440
Total Edges; 23004
All Lines of 2112: 282
Current date: 2112
Total Nodes; 23548
Total Edges; 23115
All Lines of 2201: 283
Current date: 220